In [3]:
import os
import json
import pandas as pd
import traceback

In [6]:
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
YOUR_AZURE_CREDENTIAL=os.getenv("github_api_token")

In [7]:
llm = AzureAIChatCompletionsModel(
    credential="YOUR_AZURE_CREDENTIAL",
    endpoint="https://models.github.ai/inference",
    model="gpt-4o"
)
